In [807]:
import pandas as pd 
import numpy as np 

In [808]:
df = pd.read_csv('course_lead_scoring.csv')

In [809]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [810]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [811]:
df.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score',
       'converted'],
      dtype='object')

In [812]:
categorical = ['lead_source', 'industry','employment_status', 'location','converted']

numerical = ['number_of_courses_viewed','annual_income', 'interaction_count', 'lead_score']

In [813]:
df['industry'].value_counts()

industry
retail           203
finance          200
other            198
education        187
healthcare       187
technology       179
manufacturing    174
Name: count, dtype: int64

In [814]:
df[numerical].corr()

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.031551,-0.023565,-0.004879
annual_income,0.031551,1.000000,0.048618,0.005334
interaction_count,-0.023565,0.048618,1.000000,0.009888
lead_score,-0.004879,0.005334,0.009888,1.000000


In [815]:
categoricaldf=df[categorical]
categoricaldf.fillna('NA',inplace=True)

numdf=df[numerical]
numdf.fillna(0.0,inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_25364\2271782419.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categoricaldf.fillna('NA',inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_25364\2271782419.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numdf.fillna(0.0,inplace=True)


In [816]:
df= pd.concat([categoricaldf, numdf], axis=1)
df.head()

,lead_source,industry,employment_status,location,converted,number_of_courses_viewed,annual_income,interaction_count,lead_score
0,paid_ads,NA,unemployed,south_america,1,1,79450.0,4,0.94
1,social_media,retail,employed,south_america,0,1,46992.0,1,0.80
2,events,healthcare,unemployed,australia,1,5,78796.0,3,0.69
3,paid_ads,retail,NA,australia,0,2,83843.0,1,0.87
4,referral,education,self_employed,europe,1,3,85012.0,3,0.62


In [817]:
from sklearn.model_selection import train_test_split

df_fultrain,df_test=train_test_split(df,test_size=0.2,random_state=42)
df_train,df_val=train_test_split(df_fultrain,test_size=0.25,random_state=42)

In [818]:
df_fultrain['converted'].value_counts()

converted
1    710
0    459
Name: count, dtype: int64

In [819]:
globalconvert=df_fultrain['converted'].mean()

In [820]:
df_fultrain['converted'].value_counts(normalize=True)

converted
1    0.607357
0    0.392643
Name: proportion, dtype: float64

In [821]:
df_fultrain=df_fultrain.reset_index(drop=True)
df_train=df_train.reset_index(drop=True)
df_val=df_val.reset_index(drop=True)
df_test=df_test.reset_index(drop=True)

In [822]:
y_fultrain = df_fultrain['converted']
y_train = df_train['converted']
y_val = df_val['converted']
y_test = df_test['converted'] 


In [823]:
df_fultrain.groupby('industry')['converted'].agg(['mean','count'])

,mean,count
industry,,
NA,0.557522,113
education,0.790210,143
finance,0.556886,167
healthcare,0.593333,150
manufacturing,0.621429,140
other,0.612903,155
retail,0.566265,166
technology,0.562963,135


In [824]:
for c in categorical:
    df_group=df_fultrain.groupby(c)['converted'].agg(['mean','count'])
    df_group['diff']= df_group['mean'] - globalconvert
    df_group['risk']= df_group['mean'] / globalconvert
    print(df_group)
    print()
    print()


                    mean  count      diff      risk
lead_source                                        
NA              0.669725    109  0.062368  1.102688
events          0.592965    199 -0.014392  0.976304
organic_search  0.615721    229  0.008364  1.013771
paid_ads        0.428571    210 -0.178785  0.705634
referral        0.786408    206  0.179051  1.294804
social_media    0.583333    216 -0.024023  0.960446


                   mean  count      diff      risk
industry                                          
NA             0.557522    113 -0.049835  0.917948
education      0.790210    143  0.182853  1.301064
finance        0.556886    167 -0.050470  0.916901
healthcare     0.593333    150 -0.014023  0.976911
manufacturing  0.621429    140  0.014072  1.023169
other          0.612903    155  0.005547  1.009132
retail         0.566265    166 -0.041092  0.932343
technology     0.562963    135 -0.044394  0.926907


                       mean  count      diff      risk
employment_stat

In [825]:
from sklearn.metrics import mutual_info_score

In [826]:
df_fultrain.isnull().sum()

lead_source                 0
industry                    0
employment_status           0
location                    0
converted                   0
number_of_courses_viewed    0
annual_income               0
interaction_count           0
lead_score                  0
dtype: int64

In [827]:
mutual_info_score(df_fultrain['converted'],df_fultrain['industry'])

0.011684562750165564

In [828]:
def mutual_infoscore(series):
    return mutual_info_score(series,y_fultrain)

In [829]:
df_fultrainc=df_fultrain[categorical].drop('converted',axis=1)


In [830]:
minfoscore=  df_fultrainc.apply(mutual_infoscore)
minfoscore.sort_values(ascending=False)

lead_source          0.025665
employment_status    0.013258
industry             0.011685
location             0.002253
dtype: float64

In [831]:
df_fultrain[numerical].corrwith(y_fultrain).sort_values(ascending=False)

number_of_courses_viewed    0.442068
interaction_count           0.378482
lead_score                  0.225641
annual_income               0.029612
dtype: float64

In [832]:
del df_train['converted']
del df_val['converted']
del df_test['converted']

In [833]:
train_dict= df_train.to_dict(orient='records')

In [834]:
train_dict[0]

{'lead_source': 'paid_ads',
 'industry': 'retail',
 'employment_status': 'student',
 'location': 'middle_east',
 'number_of_courses_viewed': 0,
 'annual_income': 58472.0,
 'interaction_count': 5,
 'lead_score': 0.03}

In [835]:
from sklearn.feature_extraction import DictVectorizer
dv= DictVectorizer(sparse=False)
dv.fit(train_dict)

,dtype,<class 'numpy.float64'>
,separator,'='
,sparse,False
,sort,True


In [836]:
dv.get_feature_names_out()

array(['annual_income', 'employment_status=NA',
       'employment_status=employed', 'employment_status=self_employed',
       'employment_status=student', 'employment_status=unemployed',
       'industry=NA', 'industry=education', 'industry=finance',
       'industry=healthcare', 'industry=manufacturing', 'industry=other',
       'industry=retail', 'industry=technology', 'interaction_count',
       'lead_score', 'lead_source=NA', 'lead_source=events',
       'lead_source=organic_search', 'lead_source=paid_ads',
       'lead_source=referral', 'lead_source=social_media', 'location=NA',
       'location=africa', 'location=asia', 'location=australia',
       'location=europe', 'location=middle_east',
       'location=north_america', 'location=south_america',
       'number_of_courses_viewed'], dtype=object)

In [837]:
x_train=dv.transform(train_dict)
x_train

array([[5.8472e+04, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [7.1738e+04, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        3.0000e+00],
       [8.1973e+04, 0.0000e+00, 1.0000e+00, ..., 1.0000e+00, 0.0000e+00,
        3.0000e+00],
       ...,
       [8.9042e+04, 0.0000e+00, 1.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        3.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [5.0259e+04, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        4.0000e+00]], shape=(876, 31))

In [838]:
val_dict= df_val.to_dict(orient='records')
x_val=dv.transform(val_dict)

In [839]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='liblinear',C=1.0,max_iter=1000,random_state=42)
model.fit(x_train,y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [840]:
model.intercept_

array([-0.06914728])

In [841]:
model.coef_.round(3)

array([[-0.   , -0.015,  0.034,  0.003,  0.012, -0.103, -0.025,  0.049,
        -0.02 , -0.013, -0.003, -0.009, -0.032, -0.016,  0.311,  0.051,
         0.02 , -0.012, -0.012, -0.115,  0.08 , -0.03 ,  0.004, -0.011,
        -0.011, -0.006,  0.008,  0.006, -0.033, -0.025,  0.454]])

In [842]:
df_train.shape

(876, 8)

In [843]:
model.predict(x_train)

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,

In [844]:
y_pred=model.predict_proba(x_train)

In [845]:
y_pred=model.predict_proba(x_val)[:,1]

In [846]:
y_pred.shape

(293,)

In [847]:
decision= (y_pred >=0.5)

In [848]:
(y_val == decision).mean()

np.float64(0.6996587030716723)

In [849]:
df.shape

(1462, 9)

In [850]:
df_pred = pd.DataFrame()
df_pred['probability']= y_pred
df_pred['prediction']=decision.astype(int)
df_pred['actual']= y_val
df_pred['correct']=(df_pred['prediction']==df_pred['actual'])

In [851]:
df_pred

,probability,prediction,actual,correct
0,0.611922,1,0,False
1,0.799826,1,1,True
2,0.530213,1,0,False
3,0.471315,0,0,True
4,0.570661,1,0,False
...,...,...,...,...
288,0.419342,0,0,True
289,0.710539,1,1,True
290,0.418185,0,0,True
291,0.744835,1,1,True


In [852]:
df_pred['correct'].mean()

np.float64(0.6996587030716723)

In [ ]:
C=[0.01, 0.1, 1, 10, 100]
for i in C:
    model = LogisticRegression(solver='liblinear',C=i,max_iter=1000,random_state=42)
    model.fit(x_train,y_train)
    x_train=dv.transform(train_dict)
    x_train
    y_pred=model.predict_proba(x_train)
    y_pred=model.predict_proba(x_val)[:,1]
    decision= (y_pred >=0.5)
    print(f'for regularization with {i}',(y_val == decision).mean())
    #


for regularization with 0.01 0.6996587030716723
for regularization with 0.1 0.6996587030716723
for regularization with 1 0.6996587030716723
for regularization with 10 0.6996587030716723
for regularization with 100 0.6996587030716723
